In [0]:
# -----------------------------------------------
# CHECK COMPUTE FUNCTION
#
# The function `compute_validation(recommend_dbr_classic_version=17.3, recommended_serverless_version=1)`
# checks the current Databricks compute type (All-Purpose or Serverless)
# and returns WARNINGS if the user's environment does not meet the specified requirements.
#
# Example uses:
# 1. Allow BOTH All-Purpose and Serverless:
#    compute_validation(recommend_dbr_classic_version=17.3, recommended_serverless_version=1)
#
# 2. Require ONLY All-Purpose (minimum DBR version 16.4):
#    compute_validation(recommend_dbr_classic_version=16.4, recommended_serverless_version=None)
#
# 3. Require ONLY Serverless (minimum version 3):
#    compute_validation(recommend_dbr_classic_version=None, recommended_serverless_version=3)
# -----------------------------------------------


import os
def _get_env():
    """
    Read the Databricks compute environment and extract both All Purpose or Serverless versions.

    Behavior assumptions:
      - Serverless runtime values look like 'client.X.Y'. The middle token (X) is used as the Serverless version.
      - All Purpose runtime values look like '17.3'. The full string is converted to float.
      - IS_SERVERLESS may appear as 'TRUE', 'true', or be absent. The value is uppercased.

    Returns
    -------
    dict containing:
      - is_serverless: 'TRUE' or 'FALSE'
      - current_serverless_version: int or None
      - current_dbr_version_all_purpose: float or None
    """
   
    # Note: IS_SERVERLESS may be 'TRUE' or 'true' in some envs, or absent. Uppercase the string
    is_serverless = os.environ.get("IS_SERVERLESS", "FALSE").upper()
    runtime_version = os.environ.get("DATABRICKS_RUNTIME_VERSION", "")

    # Serverless: require IS_SERVERLESS == 'TRUE', then extract the second token from 'client.X.Y'
    if is_serverless == 'TRUE':
        current_serverless_version = int(runtime_version.split(".")[1])
    else:
        current_serverless_version = None

    # All Purpose: Serverless is set to FALSE in the env variable
    if is_serverless == 'FALSE':
        current_dbr_version_all_purpose = float(runtime_version)
    else:
        current_dbr_version_all_purpose = None

    return {
        'is_serverless': is_serverless,
        'current_serverless_version':current_serverless_version, 
        'current_dbr_version_all_purpose':current_dbr_version_all_purpose
    }


def _check_serverless_only(current_serverless_version, recommended_serverless_version: int):
    """
    Validate only the Serverless requirement path and report status.
    """

    print(f"NOTE: This notebook was tested on Serverless compute version {recommended_serverless_version}. Checking compute...")

    if current_serverless_version is None:
        print(f"⚠️ WARNING: This notebook was not tested on All Purpose compute. Unexpected results or errors might occur. Please select the recommended Serverless compute version '{recommended_serverless_version}'.")
    elif current_serverless_version == recommended_serverless_version:
        print(f"✅ Serverless environment check passed: Serverless version '{current_serverless_version}' matches the recommended version.")
    else:
        print(f"⚠️ WARNING: This notebook was not tested on Serverless version '{current_serverless_version}'. Unexpected results or errors might occur. Please use Serverless version '{recommended_serverless_version}'.")


def _check_all_purpose_only(current_dbr_version_all_purpose, recommend_dbr_classic_version: float):
    """
    Validate only the All Purpose requirement path and report status.
    """

    print(f"NOTE: This notebook was tested on All Purpose compute DBR version {recommend_dbr_classic_version}. Checking compute...")

    if current_dbr_version_all_purpose is None:
        print(f"⚠️ WARNING: This notebook was not tested on Serverless compute. Unexpected results or errors might occur. Please select the recommended All Purpose compute DBR version '{recommend_dbr_classic_version}'.")
    elif current_dbr_version_all_purpose == recommend_dbr_classic_version:
        print(f"✅ All Purpose compute DBR version check passed: DBR version '{current_dbr_version_all_purpose}' matches the recommended version.")
    else:
        print(f"⚠️ WARNING: This notebook was not tested on All Purpose compute DBR version '{current_dbr_version_all_purpose}'. Unexpected results or errors might occur. Please use the recommended All Purpose compute DBR version '{recommend_dbr_classic_version}'.")


def _check_both(
    recommended_serverless_version: int, 
    recommend_dbr_classic_version: float,
    current_serverless_version: int, 
    current_dbr_version_all_purpose: float
):
    """
    Validate when either All Purpose or Serverless is acceptable and report both paths.
    """

    print(f"NOTE: This notebook can run on either All Purpose compute (recommended DBR {recommend_dbr_classic_version}) or Serverless compute (recommended version {recommended_serverless_version}).")
    print("Checking compute...")

    # All Purpose path
    if current_dbr_version_all_purpose is None:
        pass
    elif current_dbr_version_all_purpose == recommend_dbr_classic_version:
        print(f"✅ All Purpose compute DBR version check passed: DBR '{recommend_dbr_classic_version}' is the recommended version.")
    else:
        print(f"⚠️ WARNING: This notebook was not tested on All Purpose compute DBR version '{current_dbr_version_all_purpose}'. Unexpected results or errors might occur. Please use the recommended All Purpose DBR version '{recommend_dbr_classic_version}'.")

    # Serverless path
    if current_serverless_version is None:
        pass
    elif current_serverless_version == recommended_serverless_version:
        print(f"✅ Serverless environment check passed: Serverless version '{recommended_serverless_version}' is the recommended version.")
    else:
        print(f"⚠️ WARNING: This notebook was not tested on Serverless version '{current_serverless_version}'. Unexpected results or errors might occur. Please use the recommended Serverless version '{recommended_serverless_version}'.")


##
## Full function
##
def compute_validation(
    recommended_serverless_version: int = None, 
    recommend_dbr_classic_version: float = None
):
    """
    Check the Databricks compute environment and warn users when they are not running on the
    compute type or version this notebook was tested on.

    The function supports three cases:
      - Serverless only: provide `recommended_serverless_version`.
      - All Purpose only: provide `recommend_dbr_classic_version`.
      - Either compute type: provide both.

    The check compares the exact versions detected in the environment with the versions provided
    and prints warnings if they do not match. It does not raise errors for mismatches, only for
    missing inputs.

    Parameters
    ----------
    recommended_serverless_version : int or None
        Expected Serverless version this notebook was validated on.
    recommend_dbr_classic_version : float or None
        Expected All Purpose DBR version this notebook was validated on.

    Returns
    -------
    None
        Prints validation messages directly to the notebook output.
    """
    ## Add line break

    print('\n')
    # Require at least one target compute (can use both)
    if recommended_serverless_version is None and recommend_dbr_classic_version is None:
        raise ValueError(
            "Serverless version or DBR version was not specified in the function. Please specify a compute type to check."
        )

    # Get environment variable values
    env_values = _get_env()
    is_serverless = env_values["is_serverless"]   ## If it's running serverless or not
    current_serverless_version = env_values["current_serverless_version"] 
    current_dbr_version_all_purpose = env_values["current_dbr_version_all_purpose"] 


    # Perform checks: serverless only
    if recommended_serverless_version is not None and recommend_dbr_classic_version is None:
        _check_serverless_only(current_serverless_version, recommended_serverless_version)

    # Perform checks: all purpose only
    if recommended_serverless_version is None and recommend_dbr_classic_version is not None:
        _check_all_purpose_only(current_dbr_version_all_purpose, recommend_dbr_classic_version)

    # Perform checks: either path acceptable
    if recommended_serverless_version is not None and recommend_dbr_classic_version is not None:
        _check_both(
            recommended_serverless_version=recommended_serverless_version, 
            recommend_dbr_classic_version=recommend_dbr_classic_version,
            current_serverless_version=current_serverless_version,
            current_dbr_version_all_purpose=current_dbr_version_all_purpose
        )

    ## Add line break for clarity in the output
    print('\n')